# Parkinson's Freezing of Gait Prediction

#### Importing Libraries

In [1]:
import pandas as pd
import random
import numpy as np
import os
import random
from sklearn import preprocessing

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env

In [2]:
TDCSFOG_DIR = "./dataset/train/tdcsfog/"

#### Helper Functions

In [ ]:
def choose_random_file(directory):
  file_names = [f for f in os.listdir(directory) if f.endswith('.csv')]
  random_file_name = random.choice(file_names)
  return os.path.join(directory, random_file_name)

def normalize_column(df, column_name):
    min_val = df[column_name].min()
    max_val = df[column_name].max()
    df[column_name] = 2 * (df[column_name] - min_val) / (max_val - min_val) - 1

### Custom Environment

In [3]:
class ParkinsonsEnvironment(gym.Env):
    metadata = {"render_modes": ["console"]}
    
    def __init__(self, render_mode=None):
        super(ParkinsonsEnvironment, self).__init__()

        # Choose a random episode from tdcsfog data
        random_episode_dataset_path = choose_random_file(TDCSFOG_DIR)
        episode_dataframe = pd.read_csv(random_episode_dataset_path)

        # Normalize AccV, AccML, AccAP columns
        normalize_column(episode_dataframe, 'AccV')
        normalize_column(episode_dataframe, 'AccML')
        normalize_column(episode_dataframe, 'AccAP')

        
        self.episode_dataframe = episode_dataframe
        self.timestep = 0
        self.end_timestep = self.episode_dataframe.shape[0]-1
        self.render_mode = render_mode
        self.action_space = spaces.MultiBinary(3)
        self.observation_space = spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float64)

    def step(self, action):
        current_state = self.episode_dataframe.iloc[self.timestep].to_numpy()
        
        observation = current_state[1:4] # Values of AccV, AccML and AccAP
        reward = 0
        terminated = False
        truncated = False
        info = {}
        
        if self.timestep == self.end_timestep:
            terminated = True
        
        actual_action = current_state[4:] # Values of StartHesitation, Turn and Walking

        # Calculate reward
        diff = np.absolute(action - actual_action).sum()
        reward = 0
        if 1 in actual_action:
            if diff == 0:
                reward = 1/self.timestep
            else:
                reward = -1/self.timestep
        else:
            if diff == 0:
                reward = 1 / self.end_timestep
            else:
                reward = -1 / self.end_timestep
        self.timestep += 1 # Increase timestep
        
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed, options=options)

        # Choose a random episode from tdcsfog data
        random_episode_dataset_path = choose_random_file(TDCSFOG_DIR)
        episode_dataframe = pd.read_csv(random_episode_dataset_path)

        # Normalize AccV, AccML, AccAP columns
        normalize_column(episode_dataframe, 'AccV')
        normalize_column(episode_dataframe, 'AccML')
        normalize_column(episode_dataframe, 'AccAP')
        
        self.episode_dataframe = episode_dataframe
        self.timestep = 0
        self.end_timestep = self.episode_dataframe.shape[0]-1
        
        observation = self.episode_dataframe.iloc[0, 1:4].to_numpy()
        info = {}
        
        return observation, info

    def render(self):
        if self.render_mode != "console":
            return
        current_state = self.episode_dataframe.iloc[self.timestep].to_numpy()
        obs = current_state[1:4]
        print(f"AccV: {obs[0]}, AccML: {obs[1]}, AccAP: {obs[2]}")

    def close(self):
        pass

### Validating the environment

In [4]:
env = ParkinsonsEnvironment()
check_env(env, warn=True)

### Testing the environment

In [5]:
env = ParkinsonsEnvironment()

obs, _ = env.reset()
env.render()

n_steps = 10
for step in range(n_steps):
    step+=1
    print()
    print(f"Step {step}")
    action_taken = env.action_space.sample()
    print("Action:", action_taken)
    obs, reward, terminated, truncated, info = env.step(action_taken)
    done = terminated or truncated
    print("Reward:", reward)
    print()
    env.render()
    if done:
        break


Step 1
Action: [0 1 0]
Reward: -0.00020004000800160032


Step 2
Action: [1 1 1]
Reward: -0.00020004000800160032


Step 3
Action: [1 1 1]
Reward: -0.00020004000800160032


Step 4
Action: [0 0 0]
Reward: 0.00020004000800160032


Step 5
Action: [1 1 0]
Reward: -0.00020004000800160032


Step 6
Action: [0 1 0]
Reward: -0.00020004000800160032


Step 7
Action: [1 0 1]
Reward: -0.00020004000800160032


Step 8
Action: [0 0 1]
Reward: -0.00020004000800160032


Step 9
Action: [1 0 1]
Reward: -0.00020004000800160032


Step 10
Action: [1 0 1]
Reward: -0.00020004000800160032



### Applying

In [6]:
# Initializing the Environment

vec_env = make_vec_env(ParkinsonsEnvironment)

In [7]:
model = PPO("MlpPolicy", env, verbose=1,
            tensorboard_log="./parkinsons_tensorboard/")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
model.learn(total_timesteps=1_000_000, tb_log_name="PPO")

Logging to ./parkinsons_tensorboard/PPO_2
-----------------------------
| time/              |      |
|    fps             | 5653 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 4235         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0054430906 |
|    clip_fraction        | 0.00898      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.08        |
|    explained_variance   | 0.636        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0104      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00532     |
|    value_loss           | 0.000691     |
-----------------

In [9]:
model.save("ppo_model.pkl")

### Prediction

In [10]:
ppo_model = PPO.load("ppo_model.pkl")

In [11]:
test_data_dir = "./dataset/test/tdcsfog/003f117e14.csv"

In [12]:
test_df = pd.read_csv(test_data_dir)
test_df.head()

,Time,AccV,AccML,AccAP
0,0,-9.533939,0.566322,-1.413525
1,1,-9.536140,0.564137,-1.440621
2,2,-9.529345,0.561765,-1.429332
3,3,-9.531239,0.564227,-1.415490
4,4,-9.540825,0.561854,-1.429471


In [49]:
# normalize dataset

normalize_column(test_df, 'AccV')
normalize_column(test_df, 'AccML')
normalize_column(test_df, 'AccAP')

In [50]:
test_df.head()

,Time,AccV,AccML,AccAP
0,0,0.434787,0.280462,-0.326333
1,1,0.434565,0.280172,-0.329406
2,2,0.435249,0.279858,-0.328125
3,3,0.435058,0.280184,-0.326556
4,4,0.434094,0.279870,-0.328141


In [51]:
predicted_values = []

In [52]:
for i in range(len(test_df)):
    action, _ = model.predict(test_df.iloc[i, 1:4])
    predicted_values.append([f'003f117e14_{i}'] + list(action))

In [53]:
submission_df = pd.DataFrame(predicted_values, columns = ['id','StartHesitation', 'Turn', 'Walking'])

In [54]:
submission_df.head()

,id,StartHesitation,Turn,Walking
0,003f117e14_0,0.0,0.0,0.0
1,003f117e14_1,0.0,0.0,0.0
2,003f117e14_2,0.0,0.0,0.0
3,003f117e14_3,0.0,0.0,0.0
4,003f117e14_4,0.0,0.0,0.0


In [56]:
submission_df.to_csv('submission.csv', index=False)